In [83]:
import os
import cv2
import random
import math
import matplotlib.pyplot as plt
from natsort import natsorted
from uuid import uuid4

In [61]:
def load_image(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
    if image is None:
        print(f"Failed to read {image_path}")
    return image

In [70]:
def save_image_and_labels(labels, image, image_shape, i):
    h, w = image_shape[:2]
    id = str(uuid4())

    os.makedirs("train_data", exist_ok=True)
    os.makedirs("train_data/labels", exist_ok=True)
    os.makedirs("train_data/images", exist_ok=True)

    with open(f"train_data/labels/{id}-pokemon_{i}.txt", "w") as f:
        for card_id, x1, y1, x2, y2 in labels:
            x_center = ((x1 + x2) / 2) / w
            y_center = ((y1 + y2) / 2) / h
            box_width = (x2 - x1) / w
            box_height = (y2 - y1) / h
            f.write(f"{card_id} {x_center:.6f} {y_center:.6f} {box_width:.6f} {box_height:.6f}\n")

    cv2.imwrite(f"train_data/images/{id}-pokemon_{i}.png", image)

In [75]:
def draw_random_cards(blank_table, cards_dict, card_keys: list, area, num_cards, min_distance=50):
    table = blank_table.copy()
    labels = []
    placed_centers = []

    x_min, y_min, x_max, y_max = area
    for _ in range(num_cards):
        if len(cards_dict) == 0:
            break

        card_key = random.choice(list(cards_dict.keys()))
        card_id = card_keys.index(card_key)

        card_path = random.choice(list(cards_dict[card_key].keys()))
        cards_dict[card_key][card_path] -= 1
        if cards_dict[card_key][card_path] == 0:
            cards_dict[card_key].pop(card_path)
        if len(cards_dict[card_key]) == 0:
            cards_dict.pop(card_key)

        card = load_image(card_path)
        if card is None:
            continue

        if card.shape[2] == 3:
            card = cv2.cvtColor(card, cv2.COLOR_BGR2BGRA)

        target_w, target_h = 100, 140
        card = cv2.resize(card, (target_w, target_h), interpolation=cv2.INTER_AREA)

        angle = random.choice(range(-30, 31, 5))
        center = (target_w // 2, target_h // 2)
        rot_mat = cv2.getRotationMatrix2D(center, angle, 1.0)

        cos = abs(rot_mat[0, 0])
        sin = abs(rot_mat[0, 1])
        bound_w = int((target_h * sin) + (target_w * cos))
        bound_h = int((target_h * cos) + (target_w * sin))

        rot_mat[0, 2] += (bound_w / 2) - center[0]
        rot_mat[1, 2] += (bound_h / 2) - center[1]

        rotated_card = cv2.warpAffine(card, rot_mat, (bound_w, bound_h),
                                      flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT,
                                      borderValue=(0, 0, 0, 0))

        max_x = x_max - bound_w
        max_y = y_max - bound_h
        if max_x <= x_min or max_y <= y_min:
            continue

        placed = False
        for _ in range(50):  # Max attempts to find a good position
            offset_x = random.randint(x_min, max_x)
            offset_y = random.randint(y_min, max_y)

            new_center = (offset_x + bound_w // 2, offset_y + bound_h // 2)

            # Check distance to all placed cards
            if all(math.dist(new_center, existing) >= min_distance for existing in placed_centers):
                roi = table[offset_y:offset_y + bound_h, offset_x:offset_x + bound_w]
                alpha = rotated_card[:, :, 3] / 255.0
                for c in range(3):
                    roi[:, :, c] = (1 - alpha) * roi[:, :, c] + alpha * rotated_card[:, :, c]
                table[offset_y:offset_y + bound_h, offset_x:offset_x + bound_w] = roi

                labels.append((card_id, offset_x, offset_y, offset_x + bound_w, offset_y + bound_h))
                placed_centers.append(new_center)
                placed = True
                break

        if not placed:
            break  # Cannot find position without breaking min distance

    return table, labels

In [90]:
import os
from glob import glob

card_dir = "cards"
card_dict = {}

for card_name in os.listdir(card_dir):
    full_path = os.path.join(card_dir, card_name)
    if os.path.isdir(full_path):
        images = glob(os.path.join(full_path, "*"))
        card_dict[card_name] = {image: 3 for image in images}


In [91]:
card_dict = {k: v for k, v in card_dict.items() if k.startswith("sv1")}

In [92]:
card_list = natsorted(list(card_dict.keys()))

In [93]:
blank_table = load_image('table1.png')
area = (100, 100, blank_table.shape[1] - 100, blank_table.shape[0] - 100)

print(blank_table.shape[0], blank_table.shape[1])

1080 1920


In [94]:
table1 = load_image('table1.png')
table1 = (table1, (400, 200, table1.shape[1] - 100, table1.shape[0] - 100))
table2 = load_image('table2.png')
table2 = (table2, (400, 200, table2.shape[1] - 100, table2.shape[0] - 100))
table3 = load_image('table3.png')
table3 = (table3, (500, 150, 1400, table3.shape[0] - 100))
table4 = load_image('table4.png')
table4 = (table4, (500, 150, 1400, table4.shape[0] - 100))
table5 = load_image('table5.png')
table5 = (table5, (400, 200, table5.shape[1] - 100, table5.shape[0] - 100))
table6 = load_image('table6.png')
table6 = (table6, (500, 150, 1400, table6.shape[0] - 100))
table7 = load_image('table7.png')
table7 = (table7, (400, 200, table7.shape[1] - 100, table7.shape[0] - 100))

tables = [table1, table2, table3, table4, table5, table6, table7]

i = 0
while(len(card_dict) > 0):
    # table_info = random.choice(tables)
    # blank_table = table_info[0]
    # area = table_info[1]
    blank_table = load_image("white_image.png")
    area = (100, 100, blank_table.shape[1] - 100, blank_table.shape[0] - 100)
    table_with_cards, labels = draw_random_cards(blank_table=blank_table, cards_dict=card_dict, card_keys=card_list, area=area, num_cards=random.randint(3,15), min_distance=100)
    
    save_image_and_labels(labels, table_with_cards, blank_table.shape, i)
    i += 1

In [49]:
with open("classes.txt", "w") as f:
    for card in card_list:
        f.write(f"{card}\n")